In [0]:
!unzip archive.zip

Archive:  archive.zip
replace pred/res_img_259.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: pred/res_img_259.txt    
  inflating: pred/res_img_271.txt    
  inflating: pred/res_img_265.txt    
  inflating: pred/res_img_35.txt     
  inflating: pred/res_img_21.txt     
  inflating: pred/res_img_107.txt    
  inflating: pred/res_img_113.txt    
  inflating: pred/res_img_339.txt    
  inflating: pred/res_img_305.txt    
  inflating: pred/res_img_463.txt    
  inflating: pred/res_img_477.txt    
  inflating: pred/res_img_311.txt    
  inflating: pred/res_img_488.txt    
  inflating: pred/res_img_489.txt    
  inflating: pred/res_img_476.txt    
  inflating: pred/res_img_310.txt    
  inflating: pred/res_img_304.txt    
  inflating: pred/res_img_462.txt    
  inflating: pred/res_img_338.txt    
  inflating: pred/res_img_112.txt    
  inflating: pred/res_img_106.txt    
  inflating: pred/res_img_20.txt     
  inflating: pred/res_img_34.txt     
  inflating: pred/res_img_264.txt 

In [0]:
import numpy as np
import glob
import errno
import cv2
import matplotlib.path as mpltPath

In [0]:
def line_to_name(line):
    '''
    input: string with commas
    output: the last word (symbols after the last comma)
    '''
    tmp = line.split(',')
    return tmp[-1]

def line_to_box(line):
    '''
    input: string with commas, e.g. 'a,b,c,d,e,f,g,h,i'
    output: list of tuples, e.g. [('a', 'b'), ('c', 'd'), ('e', 'f'), ('g', 'h')]
    '''
    tmp = line.split(',')
    dots = []
    for i in range(0,8,2):
        a = tuple([int(tmp[i]), int(tmp[i+1])])
        dots.append(a)
    return dots    

def iou(box_0, box_1):
    '''
    input: lists of coordinates of vertices of two parallelograms
    output: intersection over union of these two parallelograms
    '''
    y_max = 0
    y_min = 100000000000
    x_max = 0
    x_min = 100000000000
    for dot in box_0:
        if (dot[0] < x_min):
            x_min = dot[0]
        if (dot[1] < y_min):
            y_min = dot[1]
        if (dot[0] > x_max):
            x_max = dot[0]            
        if (dot[1] > y_max):
            y_max = dot[1]
    for dot in box_1:
        if (dot[0] < x_min):
            x_min = dot[0]
        if (dot[1] < y_min):
            y_min = dot[1]
        if (dot[0] > x_max):
            x_max = dot[0]            
        if (dot[1] > y_max):
            y_max = dot[1]
    ys = list(range(y_min, y_max + 1))
    xs = list(range(x_min, x_max + 1))
    points = []
    for a in xs:
        for b in ys:
            points.append(tuple([a,b]))
    polygon_0 = box_0
    path_0 = mpltPath.Path(polygon_0)
    inside_0 = path_0.contains_points(points)
    points_0 = [points[i] for i in range(len(points)) if (inside_0[i] == True)]
    polygon_1 = box_1
    path_1 = mpltPath.Path(polygon_1)
    inside_1 = path_1.contains_points(points)
    points_1 = [points[i] for i in range(len(points)) if (inside_1[i] == True)]
    tmp = path_1.contains_points(points_0)
    intersection = [points_0[i] for i in range(len(points_0)) if (tmp[i] == True)]
    union = list(set().union(points_0, points_1))
    return float(len(intersection))/float(len(union))
    
def crop(image, box, ocr_list, name, number):
    name = name.lower()   
    '''
    input: image name, list of parallelogram vertices to be cropped, an auxilary list, ground true name
    
    makes a cropped version of a given image
    
    output: none
    '''
    img = cv2.imread(image)
    y_max = 0
    y_min = 100000000000
    x_max = 0
    x_min = 100000000000
    for dot in box:
        if (dot[0] < x_min):
            x_min = dot[0]
        if (dot[1] < y_min):
            y_min = dot[1]
        if (dot[0] > x_max):
            x_max = dot[0]            
        if (dot[1] > y_max):
            y_max = dot[1]
    cropped = img[y_min:y_max+1, x_min:x_max+1]
    cropped_name = 'crop/' + image[5:-4] + '_'+str(number)+'.png'
    cv2.imwrite(cropped_name, cropped)
    ocr_list.append(cropped_name + ' ' +name)

In [0]:
path = 'imgs/*.jpg'
images = glob.glob(path)
images.sort()
ocr_lines = []
unfound_words = []

In [0]:
all_gt_boxes = [] # list of ground truth boxes
all_gt_names = [] # list of ground true names (indexes correlate with the indexes of gt_boxes)

path = 'gt/*.txt'
files = glob.glob(path)
files.sort()
for file in files:
    try:
        with open(file) as f:
            gt_boxes = [] # list of ground truth boxes
            gt_names = [] # list of ground true names (indexes correlate with the indexes of gt_boxes)            
            for line in f:
                name = line_to_name(line)
                if not (name[0] == '#' and name[1] == '#' and name[2] == '#'):
                    box = line_to_box(line)
                    gt_boxes.append(box)
                    gt_names.append(name)
            all_gt_boxes.append(gt_boxes)
            all_gt_names.append(gt_names)
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

In [0]:
all_pred_boxes = [] # list of predicted boxes

path = 'pred/*.txt'
files_ = glob.glob(path)
files_.sort()
for file in files_:
    try:
        with open(file) as f:
            pred_boxes = [] # list of predicted boxes           
            for line in f:
                box = line_to_box(line)
                pred_boxes.append(box)
            all_pred_boxes.append(pred_boxes)
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

In [0]:
print(len(all_gt_boxes))
print(len(all_pred_boxes))

500
500


In [0]:
for i in range(len(all_gt_names)):
    #print()
    #print()
    print(images[i])# i is a number of image
    for j in range(len(all_gt_names[i])):
        #print()
        #print('ground true name -- ', all_gt_names[i][j])
        for k in range(len(all_pred_boxes[i])):
            res = iou(all_gt_boxes[i][j], all_pred_boxes[i][k])
            #if (res > 0): 
                #print(res, all_gt_names[i][j])

imgs/img_1.jpg
imgs/img_10.jpg
imgs/img_100.jpg
imgs/img_101.jpg
imgs/img_102.jpg
imgs/img_103.jpg
imgs/img_104.jpg
imgs/img_105.jpg
imgs/img_106.jpg
imgs/img_107.jpg


KeyboardInterrupt: ignored

In [0]:
for i in range(len(all_gt_names)): # i is a number of image
    print(images[i])
    for j in range(len(all_gt_names[i])):
        flag = 0
        for k in range(len(all_pred_boxes[i])):
            res = iou(all_gt_boxes[i][j], all_pred_boxes[i][k])
            if (res >= 0.5):
                crop(images[i], all_pred_boxes[i][k], ocr_lines, all_gt_names[i][j], j)
                flag = 1
        if (flag == 0):
            unfound_words.append(all_gt_names[i][j])

imgs/img_1.jpg
imgs/img_10.jpg
imgs/img_100.jpg
imgs/img_101.jpg
imgs/img_102.jpg
imgs/img_103.jpg
imgs/img_104.jpg
imgs/img_105.jpg
imgs/img_106.jpg
imgs/img_107.jpg
imgs/img_108.jpg
imgs/img_109.jpg
imgs/img_11.jpg
imgs/img_110.jpg
imgs/img_111.jpg
imgs/img_112.jpg
imgs/img_113.jpg
imgs/img_114.jpg
imgs/img_115.jpg
imgs/img_116.jpg
imgs/img_117.jpg
imgs/img_118.jpg
imgs/img_119.jpg
imgs/img_12.jpg
imgs/img_120.jpg
imgs/img_121.jpg
imgs/img_122.jpg
imgs/img_123.jpg
imgs/img_124.jpg
imgs/img_125.jpg
imgs/img_126.jpg
imgs/img_127.jpg
imgs/img_128.jpg
imgs/img_129.jpg
imgs/img_13.jpg
imgs/img_130.jpg
imgs/img_131.jpg
imgs/img_132.jpg
imgs/img_133.jpg
imgs/img_134.jpg
imgs/img_135.jpg
imgs/img_136.jpg
imgs/img_137.jpg
imgs/img_138.jpg
imgs/img_139.jpg
imgs/img_14.jpg
imgs/img_140.jpg
imgs/img_141.jpg
imgs/img_142.jpg
imgs/img_143.jpg
imgs/img_144.jpg
imgs/img_145.jpg
imgs/img_146.jpg
imgs/img_147.jpg
imgs/img_148.jpg
imgs/img_149.jpg
imgs/img_15.jpg
imgs/img_150.jpg
imgs/img_151.jpg
imgs/

In [0]:
with open('out.txt', 'w+') as f:
    f.writelines(ocr_lines)    

In [0]:
#ocr_lines = ocr_lines[10:]

In [0]:
#open('out.txt', 'w').close()

In [0]:
!zip -r crop.zip crop


  adding: crop/ (stored 0%)
  adding: crop/img_195_7.png (deflated 3%)
  adding: crop/img_476_18.png (deflated 0%)
  adding: crop/img_16_1.png (deflated 2%)
  adding: crop/img_480_1.png (deflated 9%)
  adding: crop/img_486_19.png (stored 0%)
  adding: crop/img_179_3.png (deflated 2%)
  adding: crop/img_56_1.png (stored 0%)
  adding: crop/img_452_3.png (stored 0%)
  adding: crop/img_121_3.png (deflated 2%)
  adding: crop/img_429_0.png (deflated 0%)
  adding: crop/img_12_2.png (stored 0%)
  adding: crop/img_125_2.png (stored 0%)
  adding: crop/img_62_3.png (deflated 2%)
  adding: crop/img_55_0.png (deflated 1%)
  adding: crop/img_20_7.png (stored 0%)
  adding: crop/img_199_2.png (deflated 0%)
  adding: crop/img_398_4.png (deflated 2%)
  adding: crop/img_454_3.png (deflated 3%)
  adding: crop/img_302_1.png (stored 0%)
  adding: crop/img_288_0.png (deflated 1%)
  adding: crop/img_201_1.png (stored 0%)
  adding: crop/img_160_3.png (stored 0%)
  adding: crop/img_159_3.png (stored 0%)
  addin